#### 1. Import pandas library

In [23]:
import pandas as pd

#### 2. Import users table:

In [55]:
users = pd.read_csv('users_table.csv')

In [56]:
users

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\r\n\r\n<p>...",0,5007,1920,-1,NaN,NaN
1,2,101,2010-07-19 14:01:36,Geoff Dalgas,2013-11-12 22:07:23,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the StackOverflow team. Find ...,25,3,0,2,37.0,NaN
2,3,101,2010-07-19 15:34:50,Jarrod Dixon,2014-08-08 06:42:58,http://stackoverflow.com,"New York, NY","<p><a href=""http://blog.stackoverflow.com/2009...",22,19,0,3,35.0,NaN
3,4,101,2010-07-19 19:03:27,Emmett,2014-01-02 09:31:02,http://minesweeperonline.com,"San Francisco, CA",<p>currently at a startup in SF</p>\r\n\r\n<p>...,11,0,0,1998,28.0,http://i.stack.imgur.com/d1oHX.jpg
4,5,6792,2010-07-19 19:03:57,Shane,2014-08-13 00:23:47,http://www.statalgo.com,"New York, NY",<p>Quantitative researcher focusing on statist...,1145,662,5,54503,35.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40320,55743,1,2014-09-13 21:03:50,AussieMeg,2014-09-13 21:18:52,NaN,NaN,NaN,0,0,0,5026902,NaN,http://graph.facebook.com/665821703/picture?ty...
40321,55744,6,2014-09-13 21:39:30,Mia Maria,2014-09-13 21:39:30,NaN,NaN,NaN,1,0,0,5026998,NaN,NaN
40322,55745,101,2014-09-13 23:45:27,tronbabylove,2014-09-13 23:45:27,NaN,United States,NaN,0,0,0,481766,NaN,https://www.gravatar.com/avatar/faa7a3fdbd8308...
40323,55746,106,2014-09-14 00:29:41,GPP,2014-09-14 02:05:17,NaN,NaN,"<p>Stats noobie, product, marketing &amp; medi...",1,0,0,976289,NaN,https://www.gravatar.com/avatar/6d9e9fa6b783a3...


#### 3. Rename Id column to userId

In [57]:
users.rename(columns={'Id': 'userId'}, inplace=True)
users


,userId,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\r\n\r\n<p>...",0,5007,1920,-1,NaN,NaN
1,2,101,2010-07-19 14:01:36,Geoff Dalgas,2013-11-12 22:07:23,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the StackOverflow team. Find ...,25,3,0,2,37.0,NaN
2,3,101,2010-07-19 15:34:50,Jarrod Dixon,2014-08-08 06:42:58,http://stackoverflow.com,"New York, NY","<p><a href=""http://blog.stackoverflow.com/2009...",22,19,0,3,35.0,NaN
3,4,101,2010-07-19 19:03:27,Emmett,2014-01-02 09:31:02,http://minesweeperonline.com,"San Francisco, CA",<p>currently at a startup in SF</p>\r\n\r\n<p>...,11,0,0,1998,28.0,http://i.stack.imgur.com/d1oHX.jpg
4,5,6792,2010-07-19 19:03:57,Shane,2014-08-13 00:23:47,http://www.statalgo.com,"New York, NY",<p>Quantitative researcher focusing on statist...,1145,662,5,54503,35.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40320,55743,1,2014-09-13 21:03:50,AussieMeg,2014-09-13 21:18:52,NaN,NaN,NaN,0,0,0,5026902,NaN,http://graph.facebook.com/665821703/picture?ty...
40321,55744,6,2014-09-13 21:39:30,Mia Maria,2014-09-13 21:39:30,NaN,NaN,NaN,1,0,0,5026998,NaN,NaN
40322,55745,101,2014-09-13 23:45:27,tronbabylove,2014-09-13 23:45:27,NaN,United States,NaN,0,0,0,481766,NaN,https://www.gravatar.com/avatar/faa7a3fdbd8308...
40323,55746,106,2014-09-14 00:29:41,GPP,2014-09-14 02:05:17,NaN,NaN,"<p>Stats noobie, product, marketing &amp; medi...",1,0,0,976289,NaN,https://www.gravatar.com/avatar/6d9e9fa6b783a3...


#### 4. Import posts table:

In [58]:
posts = pd.read_csv('posts_table.csv')
posts

,Id,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,OwnerUserId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,1,1,15.0,2010-07-19 19:12:12,23,1278.0,<p>How should I elicit prior distributions fro...,8.0,2010-09-15 21:08:26,Eliciting priors from experts,...,5.0,1,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,59.0,2010-07-19 19:12:57,22,8198.0,<p>In many different statistical methods there...,24.0,2012-11-12 09:21:54,What is normality?,...,7.0,1,8.0,88.0,2010-08-07 17:56:44,NaN,NaN,NaN,NaN,NaN
2,3,1,5.0,2010-07-19 19:13:28,54,3613.0,<p>What are some valuable Statistical Analysis...,18.0,2013-05-27 14:48:36,What are some valuable Statistical Analysis op...,...,19.0,4,36.0,183.0,2011-02-12 05:50:03,2010-07-19 19:13:28,NaN,NaN,NaN,NaN
3,4,1,135.0,2010-07-19 19:13:31,13,5224.0,<p>I have two groups of data. Each with a dif...,23.0,2010-09-08 03:00:19,Assessing the significance of differences in d...,...,5.0,2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2,NaN,2010-07-19 19:14:43,81,NaN,"<p>The R-project</p>\n\n<p><a href=""http://www...",23.0,2010-07-19 19:21:15,NaN,...,NaN,3,NaN,23.0,2010-07-19 19:21:15,2010-07-19 19:14:43,3.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,48321,2,NaN,2013-01-23 09:00:01,0,NaN,<p>you can use the matlab codes for svm and co...,19966.0,2013-01-23 09:00:01,NaN,...,NaN,0,NaN,NaN,NaN,NaN,45118.0,NaN,NaN,NaN
39996,48322,2,NaN,2013-01-23 09:09:34,3,NaN,"<p>I use <a href=""http://www.gnu.org/software/...",892.0,2013-01-23 13:13:30,NaN,...,NaN,2,NaN,892.0,2013-01-23 13:13:30,NaN,48311.0,NaN,NaN,NaN
39997,48323,2,NaN,2013-01-23 09:16:44,1,NaN,"<p>If I understand your question correctly, yo...",2020.0,2013-01-23 09:16:44,NaN,...,NaN,0,NaN,NaN,NaN,NaN,48247.0,NaN,NaN,NaN
39998,48324,2,NaN,2013-01-23 09:36:07,3,NaN,"<p>Doesn't really help you with your question,...",19914.0,2013-01-23 09:36:07,NaN,...,NaN,0,NaN,NaN,NaN,NaN,48297.0,NaN,NaN,NaN


In [27]:
#### 5. Rename Id column to postId and OwnerUserId to userId

In [59]:
posts.rename(columns={'Id': 'postId', 'OwnerUserId': 'userId'}, inplace=True)


In [ ]:
# 6. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userId,ViewCount,CommentCount

In [71]:
users_new = users.loc[:, ['userId', 'Reputation', 'Views', 'UpVotes', 'DownVotes']]

In [72]:
posts_new = posts.loc[:, ['postId', 'Score', 'userId', 'ViewCount', 'CommentCount']]

In [75]:
users_new

,userId,Reputation,Views,UpVotes,DownVotes
0,-1,1,0,5007,1920
1,2,101,25,3,0
2,3,101,22,19,0
3,4,101,11,0,0
4,5,6792,1145,662,5
...,...,...,...,...,...
40320,55743,1,0,0,0
40321,55744,6,1,0,0
40322,55745,101,0,0,0
40323,55746,106,1,0,0


In [76]:
posts_new

,postId,Score,userId,ViewCount,CommentCount
0,1,23,8.0,1278.0,1
1,2,22,24.0,8198.0,1
2,3,54,18.0,3613.0,4
3,4,13,23.0,5224.0,2
4,5,81,23.0,NaN,3
...,...,...,...,...,...
39995,48321,0,19966.0,NaN,0
39996,48322,3,892.0,NaN,2
39997,48323,1,2020.0,NaN,0
39998,48324,3,19914.0,NaN,0


#### 7. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [83]:
merged_df = pd.merge(users, posts, how = 'inner',on='userId')
merged_df = merged_df.drop_duplicates(subset='userId')

merged_df

,userId,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\r\n\r\n<p>...",0,5007,...,NaN,0,NaN,-1.0,2014-04-23 13:43:43,NaN,NaN,NaN,NaN,NaN
123,5,6792,2010-07-19 19:03:57,Shane,2014-08-13 00:23:47,http://www.statalgo.com,"New York, NY",<p>Quantitative researcher focusing on statist...,1145,662,...,15.0,5,137.0,22047.0,2013-06-07 06:38:10,2010-08-09 13:05:50,NaN,NaN,NaN,NaN
240,6,457,2010-07-19 19:04:07,Harlan,2014-08-07 19:49:44,http://www.harlan.harris.name,District of Columbia,<ul>\r\n<li>PhD in CS/AI/Machine Learning/Cogn...,114,47,...,NaN,0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
252,7,429,2010-07-19 19:04:37,Vince,2014-09-10 21:14:12,http://bioinformatics.ucdavis.edu,"Davis, CA",I'm a recent graduate of UC Davis in Economics...,56,20,...,7.0,2,7.0,88.0,2010-10-19 06:42:19,2011-06-02 11:09:11,NaN,NaN,NaN,NaN
254,8,6764,2010-07-19 19:04:52,csgillespie,2014-09-09 21:15:08,http://www.mas.ncl.ac.uk/~ncsg3/,"Newcastle, United Kingdom",<p>I'm a statistics lecturer at Newcastle Univ...,1089,604,...,5.0,1,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38957,45934,11,2014-05-21 17:13:23,jasonweiyi,2014-06-08 13:36:47,NaN,NaN,NaN,1,0,...,1.0,2,NaN,805.0,2013-05-13 04:41:41,NaN,NaN,NaN,jasonweiyi,NaN
38958,46192,36,2014-05-26 15:29:30,user1738753,2014-09-10 19:52:34,NaN,NaN,NaN,1,0,...,1.0,2,1.0,930.0,2012-10-18 15:07:40,NaN,NaN,NaN,user1738753,NaN
38959,46522,235,2014-06-01 17:25:18,Andy Blankertz,2014-09-13 18:03:07,NaN,NaN,<p>Actuary for a life insurance company.</p>\r\n,13,27,...,3.0,0,NaN,NaN,NaN,NaN,NaN,NaN,user1009703,NaN
38960,52371,221,2014-07-19 13:36:58,Karel Petranek,2014-07-20 09:32:16,NaN,NaN,NaN,2,0,...,7.0,5,8.0,NaN,NaN,2014-07-30 20:09:14,NaN,NaN,dark_charlie,NaN


#### 8. How many missing values do you have in your merged dataframe? On which columns?

In [84]:

missing_mask = merged_df.isna()
cols_with_missing = missing_mask.any(axis=0)

print("Columns with missing values:")
print(cols_with_missing[cols_with_missing == True].index.tolist())


Columns with missing values:
['DisplayName', 'WebsiteUrl', 'Location', 'AboutMe', 'Age', 'ProfileImageUrl', 'AcceptedAnswerId', 'ViewCount', 'Title', 'Tags', 'AnswerCount', 'FavoriteCount', 'LastEditorUserId', 'LastEditDate', 'CommunityOwnedDate', 'ParentId', 'ClosedDate', 'OwnerDisplayName', 'LastEditorDisplayName']


#### 9. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

In [86]:
# Fill missing values in the merged data frame with 0
merged_df.fillna('-', inplace=True)
merged_df

,userId,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\r\n\r\n<p>...",0,5007,...,-,0,-,-1.0,2014-04-23 13:43:43,-,-,-,-,-
123,5,6792,2010-07-19 19:03:57,Shane,2014-08-13 00:23:47,http://www.statalgo.com,"New York, NY",<p>Quantitative researcher focusing on statist...,1145,662,...,15.0,5,137.0,22047.0,2013-06-07 06:38:10,2010-08-09 13:05:50,-,-,-,-
240,6,457,2010-07-19 19:04:07,Harlan,2014-08-07 19:49:44,http://www.harlan.harris.name,District of Columbia,<ul>\r\n<li>PhD in CS/AI/Machine Learning/Cogn...,114,47,...,-,0,-,-,-,-,1.0,-,-,-
252,7,429,2010-07-19 19:04:37,Vince,2014-09-10 21:14:12,http://bioinformatics.ucdavis.edu,"Davis, CA",I'm a recent graduate of UC Davis in Economics...,56,20,...,7.0,2,7.0,88.0,2010-10-19 06:42:19,2011-06-02 11:09:11,-,-,-,-
254,8,6764,2010-07-19 19:04:52,csgillespie,2014-09-09 21:15:08,http://www.mas.ncl.ac.uk/~ncsg3/,"Newcastle, United Kingdom",<p>I'm a statistics lecturer at Newcastle Univ...,1089,604,...,5.0,1,14.0,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38957,45934,11,2014-05-21 17:13:23,jasonweiyi,2014-06-08 13:36:47,-,-,-,1,0,...,1.0,2,-,805.0,2013-05-13 04:41:41,-,-,-,jasonweiyi,-
38958,46192,36,2014-05-26 15:29:30,user1738753,2014-09-10 19:52:34,-,-,-,1,0,...,1.0,2,1.0,930.0,2012-10-18 15:07:40,-,-,-,user1738753,-
38959,46522,235,2014-06-01 17:25:18,Andy Blankertz,2014-09-13 18:03:07,-,-,<p>Actuary for a life insurance company.</p>\r\n,13,27,...,3.0,0,-,-,-,-,-,-,user1009703,-
38960,52371,221,2014-07-19 13:36:58,Karel Petranek,2014-07-20 09:32:16,-,-,-,2,0,...,7.0,5,8.0,-,-,2014-07-30 20:09:14,-,-,dark_charlie,-


#### 10. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [89]:
print(merged_df.dtypes)


userId                            int64
Reputation                        int64
CreationDate             datetime64[ns]
DisplayName                      object
LastAccessDate                   object
WebsiteUrl                       object
Location                         object
AboutMe                          object
Views                             int64
UpVotes                           int64
DownVotes                         int64
AccountId                         int64
Age                              object
ProfileImageUrl                  object
postId                            int64
PostTypeId                        int64
AcceptedAnswerId                 object
CreaionDate                      object
Score                             int64
ViewCount                        object
Body                             object
LasActivityDate                  object
Title                            object
Tags                             object
AnswerCount                      object


In [90]:
# Convert columns to appropriate data types

# Convert columns to appropriate data types
merged_df['CreationDate'] = pd.to_datetime(merged_df['CreationDate'])
merged_df['LastAccessDate'] = merged_df['LastAccessDate'].astype(str)
merged_df['WebsiteUrl'] = merged_df['WebsiteUrl'].astype(str)
merged_df['Location'] = merged_df['Location'].astype(str)
merged_df['AboutMe'] = merged_df['AboutMe'].astype(str)
merged_df['ProfileImageUrl'] = merged_df['ProfileImageUrl'].astype(str)
merged_df['Age'] = pd.to_numeric(merged_df['Age'], errors='coerce')
merged_df['AcceptedAnswerId'] = merged_df['AcceptedAnswerId'].astype(str)
merged_df['CreaionDate'] = merged_df['CreaionDate'].astype(str)
merged_df['ViewCount'] = pd.to_numeric(merged_df['ViewCount'], errors='coerce')
merged_df['LasActivityDate'] = merged_df['LasActivityDate'].astype(str)
merged_df['AnswerCount'] = pd.to_numeric(merged_df['AnswerCount'], errors='coerce')
merged_df['FavoriteCount'] = pd.to_numeric(merged_df['FavoriteCount'], errors='coerce')
merged_df['LastEditorUserId'] = merged_df['LastEditorUserId'].astype(str)
merged_df['ParentId'] = merged_df['ParentId'].astype(str)
merged_df['CreationDate'] = pd.to_datetime(merged_df['CreationDate'])
merged_df['LastEditDate'] = pd.to_datetime(merged_df['LastEditDate'], errors='coerce')
merged_df['CommunityOwnedDate'] = pd.to_datetime(merged_df['CommunityOwnedDate'], errors='coerce')


In [92]:
print(merged_df.dtypes)

userId                            int64
Reputation                        int64
CreationDate             datetime64[ns]
DisplayName                      object
LastAccessDate                   object
WebsiteUrl                       object
Location                         object
AboutMe                          object
Views                             int64
UpVotes                           int64
DownVotes                         int64
AccountId                         int64
Age                             float64
ProfileImageUrl                  object
postId                            int64
PostTypeId                        int64
AcceptedAnswerId                 object
CreaionDate                      object
Score                             int64
ViewCount                       float64
Body                             object
LasActivityDate                  object
Title                            object
Tags                             object
AnswerCount                     float64
